In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory


In [80]:
#Cut it the way you want
importlib.reload(AtTools)

anubis_file = "data.pkl" #'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser()
analyser.getTDC5Data(anubis_file, trigger_channel=4,bcr_channel=2, fromPKL=True, amount_of_events=10_000)
atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas.root"
data = analyser.getAtlasData(atlas_file)
analyser.correctionBCR()
#analyser.atlas_data = analyser.atlas_data[85_500:86_500]
print("Anubis", analyser.anubis_data[0].timeStamp)
print("ATLAS",analyser.atlas_data.iloc[0]["TimeStamp"])


Reading: 100%|█████████▉| 9999/10000 [00:02<00:00, 4566.43Events/s]


Anubis 1719505388.002159
ATLAS 1719498206


In [79]:
importlib.reload(AtTools)
anubis = AtTools.BCRHistogram(analyser.anubis_data, atlas=False, plot = False)
atlas = AtTools.BCRHistogram(analyser.atlas_data, atlas=True, plot = False)

window_size = 100
anubis = np.convolve(np.append(anubis, anubis[:window_size]), np.ones(window_size)/window_size, mode='valid')
atlas = np.convolve(np.append(atlas, atlas[:window_size]), np.ones(window_size)/window_size, mode='valid')
error = 100
errors = []
for offset in range(-200,200):
    errors.append(np.sum((atlas - np.roll(anubis, offset))**2))
    if np.sum((atlas - np.roll(anubis, offset))**2) < error:
        error = np.sum((atlas - np.roll(anubis, offset))**2)
        best_offset = offset

print("Best offset", best_offset)
plt.plot(atlas, label="Atlas")
#plt.plot(anubis, label="Original Anubis")
plt.plot(np.roll(anubis, best_offset), label="Offset Anubis")
plt.xlim(0, 3565)
plt.ylabel("Counts (normalized)")
plt.xlabel("BX")
plt.title("Anubis and Atlas BCR Histograms aligned - smoothed")
plt.legend()
plt.show()

plt.plot(list(range(-200,200)), errors, label="Errors")
plt.xlabel("Offset")
plt.ylabel("Sum of Square Errors")
plt.title("Sum of Square Errors vs Offset")
plt.vlines(best_offset,0, max(errors), color = "red", label=f"Best Offset: {best_offset}")
plt.ylim(0,max(errors))
plt.legend(loc = "lower left") #position

plt.show()
    

Best offset 114


In [81]:
analyser.correctBCR()
analyser.pairBCRwithEvents()

Matching:  60%|█████▉    | 86573/145275 [22:43<15:24, 63.50Events/s]   


KeyboardInterrupt: 

In [84]:
#difference
eta = []
phi = []
times = []
matches = analyser.matches
for i in range(len(matches)):
    if matches[i][1]:
        for trigger in matches[i][1]:
            #if abs(round(trigger.bcId-matches[i][0]["BCID"])) > 1000:
            #    continue
            eta.extend(matches[i][0]["mu_eta"])
            phi.extend(matches[i][0]["mu_phi"])       
            for _ in range(len(matches[i][0]["mu_eta"])):
                times.append(round(trigger.bcId+best_offset-matches[i][0]["BCID"]))


In [86]:
#3D plot
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(eta, phi, times)
ax.set_xlabel('eta')
plt.ylim(-3.2,3.2)
plt.xlim(-3.2,3.2)
ax.set_ylabel('phi')
ax.set_zlabel('time')
plt.show()

In [87]:
#Histograms
plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-np.pi, np.pi)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
# Add a color bar to show the color scale
plt.colorbar()
plt.show()

plt.hist(times, bins=range(min(times), max(times) + 2, 1))
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()